In [3]:
search_for = 128
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.011572122573852539
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 330
all cases: 866251507
type: [1, 1, 1, 1, 128] 128
cases of this type: 268435456
100000 29.426573778374628
200000 56.022084887075025
300000 65.34102532353972
400000 81.87792611982984
500000 76.8723446642737
600000 87.20608792442711
700000 79.63197769667545
800000 82.27753864111823
900000 74.91202312974
1000000 81.87155939328557
1100000 25.286644675681043
1200000 72.80506260003106
1300000 70.22300692814102
1400000 70.64268844160071
1500000 65.29073881526472
1600000 74.04516059907282
1700000 66.95308876513344
1800000 70.89823462152978
1900000 67.24838394887145
2000000 71.77874117360979
2100000 51.881300657090044
2200000 35.67060876070317
2300000 68.34067797818274
2400000 63.37188457554283
2500000 66.73928497155234
2600000 62.723246036669174
2700000 68.15973938595718
2800000 64.01552589882391
2900000 63.70057874257972
3000000 62.56467284529339

30100000 74.4219506727351
30200000 68.57745551949037
30300000 68.79076318736814
30400000 58.634596709799965
30500000 38.64635846126392
30600000 66.33154636417612
30700000 70.21914108925056
30800000 69.52128898098461
30900000 78.44664683538583
31000000 74.6797717292923
31100000 77.55694826925915
31200000 75.90946031141303
31300000 40.31847149988201
31400000 74.61533103340908
31500000 78.40299809421325
31600000 74.86385851538762
31700000 57.56088729547539
31800000 46.62097357964963
31900000 73.07338967105436
32000000 73.10394325430947
32100000 71.1252463574708
32200000 68.28771392899391
32300000 67.28259186536752
32400000 67.43489386366012
32500000 51.65289543967864
32600000 39.09935009730973
32700000 64.30409001539931
32800000 66.39206680124293
32900000 65.72614397420243
33000000 69.89793133571064
33100000 70.72305378141697
33200000 75.80968264353747
33300000 58.54668295060308
33400000 44.814836460056696
33500000 70.41781342340951
33600000 76.68654238036738
33700000 76.53195472583629
33

60500000 73.41118815484447
60600000 80.1460618300901
60700000 79.10109061204882
60800000 79.45427892945972
60900000 74.51738407796083
61000000 79.31030546622122
61100000 73.11066126256915
61200000 71.26896000127476
61300000 50.801572778109765
61400000 50.98841328409555
61500000 68.21668055084596
61600000 62.428706889369174
61700000 64.3564290031795
61800000 62.71095670839049
61900000 30.87597321056043
62000000 62.58970004848989
62100000 66.70014154904624
62200000 67.33102605889368
62300000 65.36165872456763
62400000 63.78770180658157
62500000 32.99940143894124
62600000 69.27828558845577
62700000 72.91987964839474
62800000 70.79141567216075
62900000 77.73442273097999
63000000 78.03621633676357
63100000 75.5287409368786
63200000 74.59135161535204
63300000 75.2772323643779
63400000 61.957521346824
63500000 41.357777763901765
63600000 66.36637368318092
63700000 67.20090985630888
63800000 74.67206113679215
63900000 67.39016864015815
64000000 28.483061570867374
64100000 65.37781738238247
642

90900000 39.14283082990932
91000000 51.8010130992877
91100000 64.01693934092559
91200000 57.048925015971044
91300000 27.03444290609071
91400000 61.58437492260088
91500000 65.54849961470084
91600000 38.760293411619465
91700000 58.96806106532117
91800000 61.75716181938768
91900000 67.59430019279631
92000000 66.12047303030293
92100000 71.90680651372611
92200000 73.1771331361583
92300000 69.46705546720045
92400000 72.4542385113496
92500000 76.3522589761023
92600000 72.36664689338036
92700000 70.93087259686796
92800000 67.53216783979981
92900000 63.69767172107348
93000000 49.722851605999296
93100000 44.77678544020498
93200000 67.733573527857
93300000 61.048940249034075
93400000 28.245186400599867
93500000 64.76481682022184
93600000 66.457762348606
93700000 28.42262484345635
93800000 67.75413151043033
93900000 68.65430309327346
94000000 63.10234062672962
94100000 68.40097418455362
94200000 70.38820563718036
94300000 70.38659211988818
94400000 72.71273025638348
94500000 68.33452448097991
9460

120400000 55.71097108616925
120500000 35.95048751904968
120600000 30.076844809298866
120700000 31.8140250364723
120800000 41.862988886036206
120900000 60.83021179284337
121000000 60.447982438726214
121100000 62.55849389214794
121200000 59.43863898071983
121300000 63.31825074559988
121400000 60.15630136339256
121500000 64.08689392714308
121600000 70.14413753887003
121700000 70.86401021610288
121800000 65.72652061484052
121900000 63.324157728590684
122000000 61.09152622938971
122100000 62.736070041265016
122200000 63.12487281868642
122300000 62.3901067834746
122400000 59.796557309629314
122500000 63.761127351704545
122600000 41.94886048801166
122700000 20.609721484096443
122800000 26.793844691544734
122900000 51.81636997015586
123000000 62.87581267574141
123100000 60.11155219329868
123200000 60.706022133235
123300000 61.42782906417108
123400000 63.797703099950944
123500000 65.7736436565123
123600000 63.92139909352999
123700000 67.34132504130513
123800000 67.1116698447965
123900000 60.373

...

799800000 0.25107575467402143
799900000 0.31504460388216576
best so far: 0
type: [1, 2, 16, 2, 2] 128
cases of this type: 65536
best so far: 0
type: [1, 2, 16, 4, 1] 128
cases of this type: 32768
800000000 0.3189561399894873
best so far: 0
type: [1, 2, 32, 1, 2] 128
cases of this type: 32768
best so far: 0
type: [1, 2, 32, 2, 1] 128
cases of this type: 16384
best so far: 0
type: [1, 2, 64, 1, 1] 128
cases of this type: 8192
best so far: 0
type: [1, 4, 1, 1, 32] 128
cases of this type: 4194304
800100000 0.5225292367891418
800200000 1.3814770003412862
800300000 1.0619251824450342
800400000 1.0117793532930806
800500000 0.9254625911023199
800600000 1.0151625486402764
800700000 0.9299422981001364
800800000 0.8454949875443936
800900000 0.9109062534366734
801000000 0.8983323219684886
801100000 0.8107265909444172
801200000 1.578405405434757
801300000 1.033921394928905
801400000 1.193929850669968
801500000 1.3507833630858144
801600000 0.9134402203994387
801700000 0.9880834589441717
801800000 0.

813600000 0.9231737234122892
813700000 0.91933420135446
813800000 1.281355098708376
813900000 1.1747854842735166
814000000 1.5523242792884053
814100000 1.0145660065071722
814200000 0.9505706302858704
814300000 0.8809244786328633
814400000 1.1705417401645124
814500000 1.4042420100700002
814600000 1.8095461477758965
814700000 1.8440269556759454
814800000 1.0621254775922648
814900000 1.68301442542091
815000000 1.3710874289778463
815100000 1.2869075203004738
815200000 0.9867383774824897
815300000 1.1660589837583244
815400000 1.318408360178494
815500000 0.9299539585257411
815600000 0.8308506353711547
815700000 1.5518182464258565
815800000 1.4756424567445874
815900000 1.0901904123486355
816000000 1.639368771388782
816100000 1.7628601193953635
816200000 1.6748895523120562
816300000 0.8754902103418125
816400000 0.8223416456206063
816500000 1.1242554793948498
816600000 0.9254673089377569
816700000 1.2613376472924829
816800000 1.1516166223329072
816900000 1.3496058130098725
817000000 1.413621456

841300000 0.22924075632533034
841400000 0.2869400055965119
841500000 0.17918912552575603
841600000 0.30928790163802306
841700000 0.3478437729099684
841800000 0.17324121391536856
best so far: 0
type: [2, 1, 1, 8, 8] 128
cases of this type: 2097152
841900000 0.2948963672546863
842000000 0.27985107510406704
842100000 0.33345702736004523
842200000 0.37443351242581807
842300000 0.2009503233598908
842400000 0.2150790044522385
842500000 0.23280255973991357
842600000 0.16841606943748327
842700000 0.42633582519930335
842800000 0.24874788721053004
842900000 0.2626226712505029
843000000 0.2400936920543797
843100000 0.21364444586089054
843200000 0.2612063489225957
843300000 0.294255483469875
843400000 0.1911206651397546
843500000 0.40428036360816555
843600000 0.18411999902630713
843700000 0.2177193444193661
843800000 0.22358372651537656
843900000 0.1594830168345213
best so far: 0
type: [2, 1, 1, 16, 4] 128
cases of this type: 1048576
844000000 0.27192876875689825
844100000 0.22537398586124316
8442

861400000 0.03978798209783236
861500000 0.033144450755426616
861600000 0.01961714311284953
best so far: 0
type: [2, 2, 2, 4, 4] 128
cases of this type: 131072
861700000 0.029587139574450253
best so far: 0
type: [2, 2, 2, 8, 2] 128
cases of this type: 65536
861800000 0.02313698707150022
best so far: 0
type: [2, 2, 2, 16, 1] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 1, 8] 128
cases of this type: 131072
861900000 0.022541708005439573
best so far: 0
type: [2, 2, 4, 2, 4] 128
cases of this type: 65536
862000000 0.021546449497010307
best so far: 0
type: [2, 2, 4, 4, 2] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 8, 1] 128
cases of this type: 16384
862100000 0.018084501289821333
best so far: 0
type: [2, 2, 8, 1, 4] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 8, 2, 2] 128
cases of this type: 16384
best so far: 0
type: [2, 2, 8, 4, 1] 128
cases of this type: 8192
best so far: 0
type: [2, 2, 16, 1, 2] 128
cases of this type: 8192
best so far: 0
t

866100000 0.0007812027711835174
best so far: 0
type: [8, 1, 1, 2, 8] 128
cases of this type: 32768
best so far: 0
type: [8, 1, 1, 4, 4] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 1, 8, 2] 128
cases of this type: 8192
best so far: 0
type: [8, 1, 1, 16, 1] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 1, 8] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 2, 2, 4] 128
cases of this type: 8192
866200000 0.00025720840739607806
best so far: 0
type: [8, 1, 2, 4, 2] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 8, 1] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 1, 4] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 4, 2, 2] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 4, 1] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 1, 2] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 2, 1] 128
cases of this type: 512
best so far: 0
type: [8, 1, 16, 1, 1] 128
cases of this type: 256
best so 